<a href="https://colab.research.google.com/github/GiuliaMaragno/Progetto-Sii-ML/blob/main/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from matplotlib import pyplot as plt




from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
     

train valid test 70 20 10


> Modello: ResNet 50

> target_size=224, 224

> batch_size=100

> optimizer: Adam

> epoche: 25

In [2]:
train_path ='/content/drive/MyDrive/kvasir-dataset-v2/train'
valid_path='/content/drive/MyDrive/kvasir-dataset-v2/valid'
test_path='/content/drive/MyDrive/kvasir-dataset-v2/test'

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.1, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)


In [3]:
train_set = train_datagen.flow_from_directory(directory=train_path, target_size=(224, 224), class_mode='categorical', batch_size=100)
valid_set = valid_datagen.flow_from_directory(directory=valid_path, target_size=(224, 224), class_mode='categorical', batch_size=100)
test_set = valid_datagen.flow_from_directory(directory=test_path, target_size=(224,224), class_mode='categorical', batch_size=1, shuffle=False)

Found 5703 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [4]:
resNet=tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,

)

102967424/102967424 [==============================] - 5s 0us/step


In [5]:
x = resNet.layers[-2].output
output = Dense(units=8, activation='softmax')(x)
model = Model(inputs=resNet.input, outputs=output)

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [7]:
from tensorflow.python.keras import optimizers
adam=tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name='Adam',
)

model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [8]:
step_size_train=train_set.n//train_set.batch_size
step_size_valid=valid_set.n//valid_set.batch_size

In [9]:
model.fit(x=train_set, steps_per_epoch=step_size_train, validation_data= valid_set, validation_steps=step_size_valid, epochs=25)

Epoch 1/25
57/57 [==============================] - 5035s 88s/step - loss: 0.7270 - accuracy: 0.7541 - val_loss: 647.9832 - val_accuracy: 0.1250
Epoch 2/25
57/57 [==============================] - 205s 4s/step - loss: 0.3482 - accuracy: 0.8638 - val_loss: 13.4330 - val_accuracy: 0.1250
Epoch 3/25
57/57 [==============================] - 202s 4s/step - loss: 0.3296 - accuracy: 0.8686 - val_loss: 152.6014 - val_accuracy: 0.1250
Epoch 4/25
57/57 [==============================] - 202s 4s/step - loss: 0.2768 - accuracy: 0.8922 - val_loss: 10.4016 - val_accuracy: 0.1250
Epoch 5/25
57/57 [==============================] - 202s 4s/step - loss: 0.2656 - accuracy: 0.8952 - val_loss: 3.4762 - val_accuracy: 0.1250
Epoch 6/25
57/57 [==============================] - 202s 4s/step - loss: 0.2216 - accuracy: 0.9111 - val_loss: 7.8882 - val_accuracy: 0.1250
Epoch 7/25
57/57 [==============================] - 211s 4s/step - loss: 0.1768 - accuracy: 0.9286 - val_loss: 2.1771 - val_accuracy: 0.1250
Epoch

In [ ]:
model.save("/content/drive/MyDrive/Progetto/Models/Modello ResNet50")

In [ ]:
pred = model.predict(test_set, steps = len(test_set), verbose = 2)

In [ ]:
predicted_class = np.argmax(pred, axis = 1)
true_classes = test_set.classes
class_labels= list(test_set.class_indices.keys())

from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_class, target_names=class_labels)
print(report)